# Landscape map - stage 3

This stage merges the raw longlist with the candidate data from the reference sources to create the new list.

It round-trips the data from the existing landscape list to ensure that any manual overrides are included.

In [1]:
from pathlib import Path

import petl as etl
from pipeline_utils.reference.geo import la_code_lookup

In [2]:
RAW_DATA = Path('../raw/')
DATA = Path('../data/')

Load the untagged longlist from the raw directory and augment with local authority data

In [3]:
raw = etl.fromcsv(
    RAW_DATA / 'landscape-longlist-raw.csv'
).convertnumbers(
).convert(
    'Local authority', la_code_lookup
).unpackdict(
    'Local authority'
).cache()

raw

organisation,Source,Number,LAD24CD,LAD24NM
4M Puppets,Project Grant,1,E08000021,Newcastle upon Tyne
Abdulrahman Abu - Zayd,Project Grant,1,E08000037,Gateshead
Action for Children,Project Grant,1,E06000057,Northumberland
Adam Phillips,Project Grant,3,E08000037,Gateshead
Adam Shield,Project Grant,1,E08000021,Newcastle upon Tyne


Get the list of sources in the longlist. We'll use this to update the values in the columns later on.

In [4]:
sources = tuple(raw.cut('Source').distinct().values('Source'))

sources

('IPSO', 'NPO', 'Project Grant')

Recast the longlist to convert the Source column into a column per entry, and convert any non-None values into True

In [5]:
updates = raw.recast(
    variablefield="Source",
    valuefield="Number"
).convert(
    sources,
    lambda x: True if x is not None else None
)

At this point we will also add in new data from the result of stage 2.

In [6]:
location = etl.fromcsv(RAW_DATA / 'landscape-locations.csv').lookupone('organisation', ['latitude', 'longitude'])

Set the path to the longlist

In [7]:
CULTURE_LANDSCAPE_CSV = Path('../data/culture_landscape.csv')

Create the new landscape file

In [8]:
landscape = (
    updates
    .addfield('location', lambda r: location.get(r.organisation, ()))
    .unpack('location', newfields=['latitude', 'longitude'])
)
landscape

organisation,LAD24CD,LAD24NM,IPSO,NPO,Project Grant,latitude,longitude
4M Puppets,E08000021,Newcastle upon Tyne,None,None,True,None,None
Abdulrahman Abu - Zayd,E08000037,Gateshead,None,None,True,None,None
Action for Children,E06000057,Northumberland,None,None,True,None,None
Adam Phillips,E08000037,Gateshead,None,None,True,None,None
Adam Shield,E08000021,Newcastle upon Tyne,None,None,True,None,None


Finally, write back out to the CSV file

In [9]:
landscape.tocsv(DATA / 'culture_landscape.csv')